In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 1)
         .config("spark.executor.memory", "1g")
         .getOrCreate())
sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-20 14:13:14,408 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-20 14:13:19,048 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=python-testing>

In [3]:
reads = sc.textFile('hdfs:///files/salmonella/SRR15404285.fasta').filter(lambda x: not x.startswith('>')).zipWithIndex().map(lambda x: (x[1], x[0]))
reads.take(1)

[(0,
  'TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT')]

In [4]:
reads.count()

2912309

In [57]:
len('TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT')

75

In [5]:
test_reads = reads.take(100)
test_reads

[(0,
  'TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT'),
 (1,
  'GGGTTNATCCAGACTTCATCCGGCACCGCCTCATGCAGCATCAGCACATTGCTGTAGGTCGAGTGGGTATGCCCT'),
 (2,
  'CCCAAAGATACGGGCGCAGAAAAGGCCGTCACGCTCAGGTTTGAACGTACGGTAGTTGATGGTTTCCGGCTTTTT'),
 (3,
  'CTCACGGAGAAAAGCGAAAATAAACGATTGACTCTGAAGCGGGAAAGCGTAATATGCACACCACGCGACGCTGAG'),
 (4,
  'TGACCGTTTACGCGCCTGCCGTACCGCGCAGGAAGTCCTGGATCTCATTGACCGCACCAACGCGGCAGCTTAAGA'),
 (5,
  'AACAATACATTAGTTTCCAGCGAATTGCTGCCATCTGCTGGAAAAAAGGGGCCATGAAGGCCCCCTCTTTCTGAA'),
 (6,
  'ACGGATCAGCCTGAGCGCCAGCGTGCGTATGACGCACACGGCGTCGCCTGTGCCGTTGAGCACAATGTGATATGA'),
 (7,
  'CTCATCGAGCTCACAGCACATGCGCTTTTGTGTACGGGGCTGTCACCCTGTATCGCGCGCCTTTCCAGACGCTTC'),
 (8,
  'CGGTTTCGGTTTATGCCTGATGAACCTCCCGCGCCATTCCCCGGTGCGGAATTCCGTCTTCGTCATAATCATCGG'),
 (9,
  'CGCCTGCAAGGTGCCCATCACGCCAACAACCGGTCCGACGATACCGGCGGTACGGCAGTTGCGTTCAGGCTCGAC'),
 (10,
  'GAATGAAGTGCGCAGCGTGCAGGAAAAGCTGGAAAAAGCGCTCTCGCAGGTGGCAAATGAACCTATTAACGTGTT'),
 (11,
  'TGCAGTGCATCAGGGAACAGAAATCCCCCAGAA

In [6]:
#test_reads = ['TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT',
# 'GGGTTNATCCAGACTTCATCCGGCACCGCCTCATGCAGCATCAGCACATTGCTGTAGGTCGAGTGGGTATGCCCT',
# 'CCCAAAGATACGGGCGCAGAAAAGGCCGTCACGCTCAGGTTTGAACGTACGGTAGTTGATGGTTTCCGGCTTTTT',
# 'CTCACGGAGAAAAGCGAAAATAAACGATTGACTCTGAAGCGGGAAAGCGTAATATGCACACCACGCGACGCTGAG',
# 'TGACCGTTTACGCGCCTGCCGTACCGCGCAGGAAGTCCTGGATCTCATTGACCGCACCAACGCGGCAGCTTAAGA']

In [1]:
import sys
sys.path.append('/home/ubuntu/.local/lib/python3.8/site-packages')
sys.path.append('/home/ubuntu/GenASM/build/lib.linux-x86_64-3.8') # For the homebrew GenASM bindings in C. These need to be added to the datanodes too.

import json
import pickle
import base64

import gasm # Homebrew package
import datasketch as ds
import hbase_connector

In [2]:
pool = hbase_connector.HbaseConnection(host="datanode1", port=9090)
# Table is from run_insert.sh
lsh = ds.lsh.MinHashLSH(threshold=0.8, num_perm=128, storage_config={'type': 'hbase', 'basename': b'hbase_salmonella_pos_prefix_8', 'hbase_pool': pool}, prepickle=True)

In [3]:
def create_hash(string, nperm=128):
    mh2 = ds.MinHash(num_perm=nperm)
    for i, c in enumerate(string):
        mh2.update(i.to_bytes(8, byteorder='little') + c.encode('utf8'))
    return mh2

In [10]:
%%time
for read in test_reads:
    h = create_hash(read[1], 3)
    lsh.query(h)
    #print(lsh.query(h))

CPU times: user 560 ms, sys: 39.8 ms, total: 600 ms
Wall time: 1.63 s


## GenASM docs:
```python
genasm_aligner(<reference sequence>,
               <query sequence>,
               <edit distance threshold>,
               <match score>,
               <substitution penalty>,
               <gap-opening penalty>,
               <gap-extension penalty>)
```

In [11]:
gasm.__dict__

{'__name__': 'gasmmodule',
 '__doc__': 'gasm Module',
 '__package__': '',
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader at 0x7fd2ee2c2280>,
 '__spec__': ModuleSpec(name='gasm', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7fd2ee2c2280>, origin='/home/ubuntu/GenASM/build/lib.linux-x86_64-3.8/gasm.cpython-38-x86_64-linux-gnu.so'),
 'gasmAlignment': <function gasmmodule.gasmAlignment>,
 'version': <function gasmmodule.version>,
 '__file__': '/home/ubuntu/GenASM/build/lib.linux-x86_64-3.8/gasm.cpython-38-x86_64-linux-gnu.so'}

In [12]:
# genasm_aligner(<reference sequence>, <query sequence>, <edit distance threshold>, <match score>, <substitution penalty>, <gap-opening penalty>, <gap-extension penalty>)
gasm.gasmAlignment("AATGTCC", "ATATGTCC", 3, 3, 4, 5, 1)

(1, 15, '1M1I6M', '1M1I6M', '7')

In [13]:
gasm.gasmAlignment("AATGTCC", "ATGTCCT", 3, 3, 4, 5, 1)

(3, 2, '1M1D4M2I', '1M1D4M2I', '1^A4')

In [19]:
gasm.gasmAlignment("CCATACTGAACTGACTAAC", "ACTAGAATGGCT", 100, 100, 4, 5, 1)

(6, 973, '1S1M1D2M2D3M1D2M1S2M', '2M1D2M2D3M1D5M', 'C1^A2^CT3^C2A2')

In [14]:
#%%time
debug = []

for read in test_reads:
    h = create_hash(read[1], 3)
    candidates = lsh.query(h)
    #print(candidates)
    if len(candidates) > 0:
        scores = [(cand, read, gasm.gasmAlignment(read[1], cand[1], 20, 20, 4, 5, 1)) for cand in candidates]
        #scores = [(cand[0], cand[1], cand[1]) for cand in candidates]
        scores.sort(key=lambda x: x[2][0])
        scores = list(filter(lambda x: x[2][2] != '', scores))
        #print(scores)
        if len(scores) > 0:
            if scores[0][2][3] != '':
                debug.append(scores[0])

In [15]:
debug #(candidate, read, comparison)

[((4099898,
   'AGACCAGAACCTCACGGAGAAAAGCGAAAATAAACGCTTGACTCTGAAGCGGGAAAGCGTAATATGCACACCCCG'),
  (3,
   'CTCACGGAGAAAAGCGAAAATAAACGATTGACTCTGAAGCGGGAAAGCGTAATATGCACACCACGCGACGCTGAG'),
  (12, 1237, '10I26M1S35M1S2M', '10I65M', '26A35A2')),
 ((4627367,
   'TGACCGTTTACGCGCCTGCCGTACCGCGCAGGAAGTCCTGGATCTCATTGACCGCACCAACGCGGCAGCTTAAGA'),
  (4,
   'TGACCGTTTACGCGCCTGCCGTACCGCGCAGGAAGTCCTGGATCTCATTGACCGCACCAACGCGGCAGCTTAAGA'),
  (0, 1500, '75M', '75M', '75')),
 ((3451544,
   'AACAATACATTAGTTTCCAGCGAATTGCTGCCATCTGCTGGAAAAAAGGGGCCATGAAGGCCCCCTCTTTCTGAA'),
  (5,
   'AACAATACATTAGTTTCCAGCGAATTGCTGCCATCTGCTGGAAAAAAGGGGCCATGAAGGCCCCCTCTTTCTGAA'),
  (0, 1500, '75M', '75M', '75')),
 ((4406970,
   'ACGGATCAGCCTGAGCGCCAGCGTGCGTATGACGCACACGGCGTCGCCTGTGCCGTTGAGCACAATGTGATATGA'),
  (6,
   'ACGGATCAGCCTGAGCGCCAGCGTGCGTATGACGCACACGGCGTCGCCTGTGCCGTTGAGCACAATGTGATATGA'),
  (0, 1500, '75M', '75M', '75')),
 ((2799395,
   'CTCATCGAGCTCACAGCACATGCGCTTTTGTGTACGGGGCTGTCACCCTGTATCGCGCGCCTTTCCAGACGCTTC'),
  (7,
   'CT

In [16]:
table = {}

for v in debug:
    candidate_index, candidate = v[0]
    read_index, read = v[1]
    comparison = v[2]
    
    for i, base in enumerate(read):
        index = candidate_index + i
        if not index in table:
            table[index] = {}
            
        if not base in table[index]:
            table[index][base] = {}
            
        if not (read_index, i) in table[index][base]:
            table[index][base] = []
            
        table[index][base].append((read_index, i, comparison))

In [24]:
reference = sc.textFile("hdfs:///files/salmonella/assembledASM694v2").filter(lambda x: not x.startswith('>')).zipWithIndex().map(lambda x: (x[1]*80, x[0]))
reference.take(5)

[(0,
  'AGAGATTACGTCTGGTTGCAAGAGATCATGACAGGGGGAATTGGTTGAAAATAAATATATCGCCAGCAGCACATGAACAA'),
 (80,
  'GTTTCGGAATGTGATCAATTTAAAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAATACAAGACAGAC'),
 (160,
  'AAATAAAAATGACAGAGTACACAACATCCATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCACAGGT'),
 (240,
  'AACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGAACAGTGCGGGCTTTTTTTTCGACCAGAGA'),
 (320,
  'TCACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCG')]

In [62]:
'A' * 75

'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [68]:
h1 = create_hash('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', nperm=128)
h2 = create_hash('TTTTTTTTTTTTTTTTTTTTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', nperm=128)
h1.jaccard(h2)

0.5546875

In [58]:
len('AGAGATTACGTCTGGTTGCAAGAGATCATGACAGGGGGAATTGGTTGAAAATAAATATATCGCCAGCAGCACATGAACAA')

80

In [18]:
gasm.gasmAlignment('AGAGATTACGTCTGGTTGCAAGAGATCATGACAGGGGGAATTGGTTGAAAATAAATATATCGCCAGCAGCACATGAACAA', 'AGAGAAAACGTCTGGTTGCAAGAGATCATGACAGGGGGTTTTGGTTGAAAATAAATATATCGCCAGCAGCACATGAACAA', 20, 20, 4, 5, 1)

(4, 1504, '5M2S31M2S40M', '80M', '5TT31AA40')

In [13]:
base64.b64decode('aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAQ==')

b'hbase_salmonella_pos_prefix_7_bucket_\x00\x10'

In [10]:
with pool as c:
    t = c.getTableNamesByNamespace('')
t

[TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAQ'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAg'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAw'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8ABA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2tleXM'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAB'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAC'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAD'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcH

In [7]:
%%time
candidate = list(lsh.query(create_hash('AACAAGTTTCGGAATGTGATCAATTTAAAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAATA')))
candidate.sort(key=lambda x: x[0]) 
candidate = candidate[0:7]
candidate

CPU times: user 5.87 ms, sys: 166 µs, total: 6.04 ms
Wall time: 32.5 ms


[(75,
  'AACAAGTTTCGGAATGTGATCAATTTAAAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAATA')]

In [23]:
%%time
scores = [(cand, gasm.gasmAlignment('TTCAAGTTTCGTAATGTGATCTATTTAAAAATTTATTGACTTATGCGGGCAGATACTTTAACTAATATAGGAATA', cand[1], 20, 20, 4, 5, 1)) for cand in candidate]
scores.sort(key=lambda x: x[1][0])
scores = list(filter(lambda x: x[1][2] != '', scores))
scores

CPU times: user 269 µs, sys: 60 µs, total: 329 µs
Wall time: 333 µs


[((75,
   'AACAAGTTTCGGAATGTGATCAATTTAAAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAATA'),
  (6, 1356, '2S9M1S9M1S21M1S18M1S12M', '75M', 'TT9T9T21T18T12'))]

In [ ]:
ref = reference.filter(lambda x: x[0] >= 4099898).sortByKey(lambda x: x[0]).take(1)[0]
ref

In [46]:
keys = [key for key in table.keys() if key >= 4099920 and key < 4099920 + 81]
for key in keys:
    offset = key - ref[0]
    reference = ref[1][offset]
    value = list(table[key].keys())[0]
    print(f"{key}: {value}/{reference}")

4099920: A/A
4099921: A/G
4099922: C/C
4099923: G/G
4099924: A/A
4099925: T/A
4099926: T/A
4099927: G/A
4099928: A/T
4099929: C/A
4099930: T/A
4099931: C/A
4099932: T/C
4099933: G/G
4099934: A/C
4099935: A/T
4099936: G/T
4099937: C/G
4099938: G/A
4099939: G/C
4099940: G/T
4099941: A/C
4099942: A/T
4099943: A/G
4099944: G/A
4099945: C/A
4099946: G/G
4099947: T/C
4099948: A/G
4099949: A/G
4099950: T/G
4099951: A/A
4099952: T/A
4099953: G/A
4099954: C/G
4099955: A/C
4099956: C/G
4099957: A/T
4099958: C/A
4099959: C/A
4099960: A/T
4099961: C/A
4099962: G/T
4099963: C/G
4099964: G/C
4099965: A/A
4099966: C/C
4099967: G/A
4099968: C/C
4099969: T/C
4099970: G/C
4099971: A/C
4099972: G/G


AGCGAAAATAAACGCTTGACTCTGAAGCGGGAAAGCGTAATATGCACACCCCGCGCCGCTGAGAAAAAGCGAAGCGGCAC

AACGATTGACTCTGAAGCGGGAAAGCGTAATATGCACACCACGCGACGCTGAG

In [40]:
len('AACGATTGACTCTGAAGCGGGAAAGCGTAATATGCACACCACGCGACGCTGAG')

53

In [30]:
a = 20
a.to_bytes(8, byteorder='little') + 'c'.encode('utf8')

b'\x14\x00\x00\x00\x00\x00\x00\x00c'

In [10]:
lines = """aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAA
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAB
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAC
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAD
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAE
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAF
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAG
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAH
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAI
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAJ
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAK
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAL
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAM
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAN
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAO
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAP
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAQ
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAR
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAS
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAT
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAU
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAV
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAW
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAX
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAY
aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4Xzdfa2V5cw""".split('\n')